In [2]:
from common import *
from matrix import *
from helper.weighted import *
%matplotlib inline
canvas(16, 12)

In [4]:
from models.thealg import alg

In [5]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
n_days = len(days)
days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
#days_a, days_b = set(days[:n_days // 2]), set(days[n_days // 2:])
tr, cv, ep = df.Day.isin(days_a), df.Day.isin(days_b), df.Day.isin(days_c)
#tr, cv = df.Day.isin(days_a), df.Day.isin(days_b)

In [ ]:
raise Stop

In [7]:
duse = dfh # <=============
rawfeats = x_feats # <============
def_matrices(globals(), duse, 10000)
for xname in rawfeats:
    X = duse.pivot('Day', 'Stock', xname)
    Xa = X.fillna(0)
    Xm = X.fillna(X.mean())
    Xpct = X.pct()
    Xpcta = Xpct.fillna(0)
    Xpctm = Xpct.fillna(0.5)
    for ext in ['', 'a', 'm', 'pct', 'pcta', 'pctm']:
        globals()[xname.upper() + ext] = locals()['X' + ext]

In [8]:
def mse(Y_, st):
    return (W[st] * (Y_ - Y[st]) ** 2).sum().sum() / W[st].sum().sum()
def rmse(Y_, st):
    return np.sqrt(mse(Y_, st))
def prss(Y_, st):
    return mse(Y_, st) / mse(0, st)
def lbss(Y_, st):
    return prss(Y_, st) * 0.500944
def metric(Y_, st):
    print('(' + ', '.join('{:.6f}'.format(round(func(Y_, st), 6)) for func in [rmse, prss, lbss]) + ')')

In [9]:
Xa = pd.concat({x: globals()[x.upper() + 'a'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)
Xpcta = pd.concat({x: globals()[x.upper() + 'pcta'] for x in rawfeats}, axis=1)#.swaplevel(axis=1).sort_index(axis=1)

In [10]:
Xuse = Xpcta #.loc[:, (ss, 'x4')]

#### my alg (old, reference)

In [10]:
save_dir = big_dir # <===============
fnames = 'x_.sgnint x_.log.sgnint x3.sgnint x3.log.sgnint'.split() # <==============
Xfeats = []
for fname in fnames:
    print(fname, end=' ')
    _Xfeat = pd.read_hdf(save_dir + '/data/features/' + fname + '.hdf5')
    Xfeats.append(_Xfeat)
Xfeat = pd.concat(Xfeats, axis=1)

x_.sgnint x_.log.sgnint x3.sgnint x3.log.sgnint 

In [11]:
#keep = [i for i, s in enumerate(Xfeat.columns) if len(s) <= len('x_.sgnint(0000/)')] # <================
filt = index_nvdf(dh, 246) # <==============
Xfeat = Xfeat[filt]
#Xfeat = Xfeat.iloc[:, keep]

In [12]:
Xfeat = make_stock_wide(Xfeat)

In [13]:
#Xfeat.fillna(0, inplace=True)
Xfeat = Xfeat.pct()
Xfeat.fillna(0.5, inplace=True)
Xfeat *= 2
Xfeat -= 1

In [14]:
Ya = make_stock_wide(dh.y).fillna(0)
Wa = make_stock_wide(dh.Weight).fillna(0)

### testing! / easy

In [6]:
import gc
def colname_append(df, suffix):
    df.columns += suffix
    return df
pd.DataFrame.colname_append = colname_append
makesw = make_stock_wide

In [7]:
Ya = make_stock_wide(dh.y).fillna(0)
Wa = make_stock_wide(dh.Weight).fillna(0)

In [10]:
_demean = lambda x: x - x.wmean(Wa)

#### features

In [11]:
_day = dh[['Day']]
Xfd = dh[['Day']].pipe(lambda d: (d - d.min()) / (d.max() - d.min())).pipe(makesw).pipe(_demean)

In [ ]:
save_dir = big_dir + '/data/features/'
fnames = 'x_.mmsgnco x_.log.mmsgnco x3.mmsgnco x3.log.mmsgnco'.split()
raw_, pct_ = [1, 1, 1, 1], [1] * 4
_Xfsi = []
for fname, r_, p_ in zip(fnames, raw_, pct_):
    print(fname, end=' ')
    _X = pd.read_hdf(save_dir + '64/' + fname + '.hdf5').pipe(make_stock_wide)
    if r_:
        _Xfsi.append(_X.astype(np.float32))
    _X = _X.wpct(Wa, [-1, 1]).colname_append('.wpct')
    if p_:
        _Xfsi.append(_X.astype(np.float32))
    del _X
Xfsi = pd.concat(_Xfsi, axis=1)
del _Xfsi
gc.collect()

x_.mmsgnco 

In [7]:
#keep = [i for i, s in enumerate(Xfeat.columns) if len(s) <= len('x_.sgnint(0000/)')] # <================
#xf1 = dh[x_feats + x3feats]
#xf2 = dh[x_feats + x3feats].replace(0, np.exp(-200)).pipe(np.log)
xf1 = pd.read_hdf(big_dir + '/data/features/' + 'x_.mintco[2,1]' + '.hdf5')
xf2 = pd.read_hdf(big_dir + '/data/features/' + 'x_.log.mintco[2,1]' + '.hdf5')
#Xfeat = Xfeat.iloc[:, keep]
xf1.columns += '.wpct'
xf2.columns += '.wpct'
Xf1 = make_stock_wide(xf1).wpct(W.fillna(0)) * 2 - 1
Xf2 = make_stock_wide(xf2).wpct(W.fillna(0)) * 2 - 1

/home/ec2-user/Notebooks/g-research-ffc2018/helper/weighted.ipynb:13: RuntimeWarning: invalid value encountered in true_divide
  },


In [12]:
Xa = pd.concat([Xfd, Xf1, Xf2], axis=1).fillna(0)

In [16]:
sfilt = ~stock_nvdf(dh, 246) # <==============
Xtest = Xa.iloc[:, np.concatenate([sfilt] * Xa.columns.levels[0].shape[0])]
Ytest = Ya.T[sfilt].T
Wtest = Wa.T[sfilt].T
tr, cv, ep = Ytest.index.isin(days_a), Ytest.index.isin(days_b), Ytest.index.isin(days_c)
scores_, models_ = alg(Xtest[tr], Ytest[tr], Wtest[tr],
                     Xtest[cv], Ytest[cv], Wtest[cv],
                     Xtest[ep], Ytest[ep], Wtest[ep], intercept=False)

0.8149930222812283 0.8543656304886941 0
()()

[99]0.........10.........20.........30.........40.........50.........60.........70.........80.........90........
0.7487223508398001 0.7784317114231387 1
(x_.log.sgnint(0/4).wpct)(63,)

[98]0.........10.........20.........30.........40.........50.........60.........70.........80.........90.......
0.748661501543552 0.7806330917919561 2
(x_.log.sgnint(0/4).wpct, x_.log.sgnint(3/4).wpct)(63, 87)

[97]0.........10.........20.........30.........40.........50.........60.........70.........80.........90......


In [46]:
np.save(top_dir + 'data/progress/niceee.npy', models[0,])

#### small

In [ ]:
from sklearn.linear_model import LinearRegression as LR

def do_lr(X, Y, W):
    lr_list = []
    # assert the columns of the input can be indexed by range(0, 3023)
    for k in range(N_STOCK):
        x, y, w = X[k], Y[k], W[k]
        if w.sum() == 0:
            lr_list += [None]
            continue
        lr = LR(fit_intercept=True)
        # don't forget x1 and x2 have NaNs for valid y. try setting W=0 for when y and W does not exist and then filling in NaNs
        lr = lr.fit(x, y, w)
        lr_list += [lr]
    return lr_list

In [66]:
lr_list = do_lr(Xuse[tr], Ya[tr], Wa[tr])

In [51]:
Yx = np.stack([(lr.predict(Xuse[k]) if lr is not None else np.zeros(Y.shape[0]))
               for lr, k in zip(lr_list, range(N_STOCK))], axis=1)
Yx = Yx + Y * 0

In [52]:
st = cv
Yxm = Yx.where(Y[tr].count() > 50, Y[tr].mean().fillna(0), axis=1)

#### big

In [20]:
bigX = Xpcta.stack(level=0)
bigY = Ya.stack(level=0)
bigW = Wa.stack(level=0)

In [24]:
lr = LR()
lr = lr.fit(bigX, bigY, bigW)

In [28]:
Yx = np.stack([(lr.predict(Xpcta[k]) if lr is not None else np.zeros(Y.shape[0]))
               for k in range(N_STOCK)], axis=1)
Yxm = Yx + Y * 0

### metric test sees

In [53]:
metric(Yxm[tr], tr)

(2.869601, 0.748053, 0.374732)


In [54]:
metric(Yxm[cv], cv)

(2.927475, 0.795920, 0.398711)


In [21]:
metric(Y[tr].wmean(W[tr]), cv)

(2.951820, 0.809212, 0.405370)


In [22]:
metric(Y[tr].mean(), cv)

(2.961588, 0.814577, 0.408057)


In [19]:
metric(0, cv)

(3.281396, 1.000000, 0.500944)


In [18]:
metric(Y[cv]*.225, cv)

(2.543082, 0.600625, 0.300879)


### stop

In [ ]:
raise Stop

In [ ]:
k = 2
plt.scatter(Xuse[k], Ya[k]);
plt.plot(Xuse[k], Yx[k]);
plt.xlim(-0.0001, 0.0004);
plt.ylim(-0.0002, 0.0001);